In [35]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %load_ext autoreload
%autoreload 2
from utils.core import Load_data
import pandas as pd
import numpy as np
from Model.preprocessing.pipelines import Data_Process
from Model.preprocessing.datasetManager import DatasetManger
from Model.base.train import cross_validation
from Model.base.model import make_model
from Model.base.logger import LoggerManager
import tensorflow as tf
from Model.base.evaluator import EvalutionClassifer, Drow, Drow_mean, Save_fig
from Model.base.augmentation import Augmentation, Strategies
from ydata_profiling import ProfileReport
import dataframe_image as dfi

In [36]:
column_interest = [
    'ti-rads_level',
    'original_ngtdm_Coarseness',
    'wavelet-HH_glcm_MaximumProbability',
    'wavelet-LH_glszm_LargeAreaHighGrayLevelEmphasis',
    'sex',
    'age',
    'size_x',
    'ti-rads_margin',
    'ti-rads_echogenicfoci',
    'wavelet-LH_glcm_SumEntropy',
    'original_firstorder_Kurtosis',
    'wavelet-LL_glszm_LargeAreaLowGrayLevelEmphasis',
    'wavelet-HL_firstorder_Energy',
    'size_y',
    'ti-rads_shape',
    'wavelet-HH_glrlm_RunLengthNonUniformityNormalized',
    'size_z',
    'wavelet-LL_glcm_MCC',
    'wavelet-HH_firstorder_Variance',
    'wavelet-HL_glszm_HighGrayLevelZoneEmphasis',
    'wavelet-HH_glszm_LargeAreaHighGrayLevelEmphasis',
    'original_firstorder_InterquartileRange',
    'wavelet-HL_glrlm_ShortRunLowGrayLevelEmphasis',
    'wavelet-HL_glszm_LargeAreaHighGrayLevelEmphasis',
    'original_firstorder_Skewness',
    'wavelet-LH_gldm_DependenceEntropy',
    'original_glszm_LargeAreaLowGrayLevelEmphasis',
    'wavelet-HH_glrlm_RunLengthNonUniformity',
    'original_shape2D_Sphericity',
    'wavelet-HL_firstorder_Median'
]

In [37]:
dataset_pd = pd.read_csv(
    "../dataset/FullDatasetAndWavlet.csv", index_col="annot_id")
dataset_pd.loc["125_"] = dataset_pd.loc["125_"].fillna(0.9)

In [38]:
X = dataset_pd[column_interest]


Y = dataset_pd['histopath_diagnosis'].astype(np.float32)

In [39]:
categorical_features = ["sex"]  # remvoed by mrmr "location"

# ["ti-rads_level", "size_x", "size_y", "size_z", "age" ]
numerical_features = ["ti-rads_level", "age"]

Ti_rads_featurea = ["ti-rads_margin", "ti-rads_shape",
                    "ti-rads_echogenicfoci"]  # removed by mrmr "ti-rads_composition", "ti-rads_echogenicity"

numerical_features = list(X[X.columns[~X.columns.isin(
    categorical_features + Ti_rads_featurea)]].columns)

In [40]:
copy_X = X.copy()
copy_X["target"] = Y

In [41]:
corr = copy_X[numerical_features +
              Ti_rads_featurea].corrwith(copy_X["target"]).sort_values()
corr.index.names =["features"]
dfi.export(corr.to_frame("PCC").style.background_gradient(cmap='coolwarm'),'df_styled.png')